In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from collections import Counter
from konlpy.tag import Okt
import re
import os

okt=Okt()

data = pd.read_csv('./data/total_reault.csv') #원본 데이터
#tf불러오기
with open('./data/questionsTF.pickle', 'rb') as f:
    tf = pickle.load(f)
#idf불러오기
with open('./data/questionsIDF.pickle', 'rb') as f:
    idf = pickle.load(f)
#tfidf불러오기
with open('./data/questionsTFIDF.pickle', 'rb') as f:
    tfidf = pickle.load(f)
    

In [3]:
class Similarity():
   
    def __init__(self,new_text):
        self.new_text=new_text

    def get_new_text_tfdtm(self):
        # 새로운 글 명사 추출 후 TF구하기
        self.new_nouns=okt.nouns(self.new_text) # new_text 명사추출
        self.new_tf=pd.Series(Counter(self.new_nouns)) # new_text TF구하기
        self.new_text_filter=[i for i in self.new_tf.index if i in idf.index] # 새로 생긴 단어(idf에 없는 단어) 없애기
        self.new_tf=self.new_tf[self.new_text_filter]
        return self.new_tf
    
    def get_new_text_tfidf(self):
        # 새로운 글 tfidf구하기
        self.new_tfidf=self.new_tf*idf
        self.new_tfidf=self.new_tfidf.fillna(0)
        self.new_tfidf=self.new_tfidf.values.reshape(1,-1)
        return self.new_tfidf
    
    def get_similarity3(self):
        self.get_new_text_tfdtm()
        self.get_new_text_tfidf()

        # # 유사한 글 30개 추출(유사도 내림차순으로 정렬)
        self.sim_res=[cosine_similarity(self.new_tfidf,tfidf.loc[i].values.reshape(1,-1)).flatten().tolist() for i in tfidf.index]
        self.sim_res_dict=[(idx,i[0]) for idx, i in enumerate(self.sim_res)]
        self.sim30_sort=sorted(self.sim_res_dict, key=lambda x: x[1], reverse=True)[:10] # 유사한 문서 30개 (행번호,코사인유사도 값)
        self.sim30_sort_index=[i[0] for i in self.sim30_sort] #  유사한 행번호 30개, 내림차순 정렬
        self.sim30_sort_score=[i[1] for i in self.sim30_sort] #  유사도 점수 30개, 내림차순 정렬
        #저장
        filename=' '.join(re.findall('[가-힣0-9a-zA-Z]+',self.new_text)) #파일이름(한글,숫자, 영어)로 제한, 특수문자 제외
        data.iloc[self.sim30_sort_index].to_csv(f'./data/질문_similarity30_{filename[:10]}.csv')
        print(f'"{os.getcwd()}"폴더에 유사한 글 30개가 "./data/질문_similarity30_{filename[:10]}.csv" 파일명으로 저장되었습니다.')
        return data.iloc[self.sim30_sort_index]
    
    def view_similarity3(self):
        
        # 새로운글과 유사한 문장 3개 보여주기
        self.df=self.get_similarity3()
        print('==============================================================================================')
        print(f'   >>> 질문(Q) : {self.new_text}')
        print('==============================================================================================')
        
        # -----<수정>  질문에 대한 응답 여러개 처리
        CNT=0
        TMP=None
        self.result=[]
        tmp=''

        # 질문에 해당하는 답변 
        for idx,i in enumerate(self.df.index):
            if CNT>3:
                break
            if self.df['title'][i]!=TMP:
                self.result.append(tmp)
                CNT+=1
                유사도=self.sim30_sort_score[idx]
                날짜=self.df['날짜'][i]
                제목=self.df['title'][i]
                질문=self.df['questions'][i]
                응답=self.df['answer'][i]
                tmp={
                    'CNT':CNT,
                    '유사도':유사도,
                    '날짜':날짜,
                    '제목':제목,
                    '질문':질문,
                    '응답':[응답]
                }
                TMP=self.df['title'][i]
            else:
                응답=self.df['answer'][i]
                tmp['응답'].append(응답)
        
        # 유사한 질문 3가지 출력
        for i in self.result[1:]:
            #print(f'{i['CNT']}')
            print('[{}] 유사도:{}'.format(i['CNT'], i['유사도']))
            print()
            print('   ▶ 제목: {}'.format(i['제목']))
            print()
            print('   ▶ 질문')
            print(i['질문'])
            print()
            print('   ▶ 응답')
            for idx, j in enumerate(i['응답'],1):
                print(f'<응답{idx}> {j}')
                print()
            print('--------------------------------------------------------------------------------------')
        
if __name__=='__main__':
    Q='''안녕하세요  갭투자 컨설팅 회사로 인해 계약한 집주소로 전세대출 및 
    담보 대출 진행을 하였었는데 현재 전세사기 조사는 받았으나 이번 주중에 담보대출 진행한 개인대부업체에서
    공문서위조로 고소장 소송예정인데 고소장이 들어가면 바로 영장발부 되어 체포하러 오나요? 
    아니면 연락으로 조사 받으러 오라고 하여 바로 수감 되나요? 준비를 좀 하고싶어 변호사 상담 요청 합니다 ㅜ ㅜ ㅜ'''
    s_text=Similarity(Q)
    s_text.view_similarity3()

"C:\Users\hmh\Documents\py\_프로젝트\최종_문서 유사도\최종문서유사도 - 1"폴더에 유사한 글 30개가 "./data/질문_similarity30_안녕하세요 갭투자 .csv" 파일명으로 저장되었습니다.
   >>> 질문(Q) : 안녕하세요  갭투자 컨설팅 회사로 인해 계약한 집주소로 전세대출 및 
    담보 대출 진행을 하였었는데 현재 전세사기 조사는 받았으나 이번 주중에 담보대출 진행한 개인대부업체에서
    공문서위조로 고소장 소송예정인데 고소장이 들어가면 바로 영장발부 되어 체포하러 오나요? 
    아니면 연락으로 조사 받으러 오라고 하여 바로 수감 되나요? 준비를 좀 하고싶어 변호사 상담 요청 합니다 ㅜ ㅜ ㅜ
[1] 유사도:0.9191946361622979

   ▶ 제목: 전세사기 변호사 선임

   ▶ 질문
안녕하세요갭투자 컨설팅 회사로 인해 계약한 집주소로 전세대출 및 담보 대출 진행을 하였었는데 현재 전세사기 조사는 받았으나 이번 주중에 담보대출 진행한 개인대부업체에서 공문서위조로 고소장 소송예정인데 고소장이 들어가면 바로 영장발부 되어 체포하러 오나요? 아니면 연락으로 조사 받으러 오라고 하여 바로 수감 되나요? 준비를 좀 하고싶어 변호사 상담 요청 합니다 ㅜ ㅜ ㅜ

   ▶ 응답
<응답1> 안녕하세요. 서울지방변호사회 소속으로 지식iN 법률상담을 진행하고 있는 권희영 변호사입니다.?1. 대부업체가 공문서를 위조한 혐의로 현재 고소장을 경찰에 제출한 상태인가요???2. 해당 고소장이 경찰에 접수가 되면,경찰에 고소인이 출석하여 해당 고소장을 진술하고,경찰은 해당 내용을 바탕으로 피고소인을 소환하여 조사를 하게 됩니다.?(고소장이 접수된다고 바로 체포영장이 발부되지는 않아요)???권희영 변호사 드림.

--------------------------------------------------------------------------------------
[2] 유사도:0.2990517416908143

   ▶ 제목

## test

In [4]:
Q='''500으로 신혼부부전세대출이나LH전세대출이나 융자(담보대출)받을 수 잇나요?현ㅈㅐ 시댁에서 살고 있습니다5개월된 아이가 잇고 뱃속에 임신중인데저도 모르게 스트레스가 쌓여자꾸 출혈이 있네요.. 한시라도 빨리나가고 싶은데 방법있을까요?'''
s_text=Similarity(Q)
s_text.view_similarity3()

"C:\Users\hmh\Documents\py\_프로젝트\최종_문서 유사도\최종문서유사도 - 1"폴더에 유사한 글 30개가 "./data/질문_similarity30_500으로 신혼부부.csv" 파일명으로 저장되었습니다.
   >>> 질문(Q) : 500으로 신혼부부전세대출이나LH전세대출이나 융자(담보대출)받을 수 잇나요?현ㅈㅐ 시댁에서 살고 있습니다5개월된 아이가 잇고 뱃속에 임신중인데저도 모르게 스트레스가 쌓여자꾸 출혈이 있네요.. 한시라도 빨리나가고 싶은데 방법있을까요?
[1] 유사도:0.7946995587839075

   ▶ 제목: 500으로 신혼부부전세대출이나LH전세대출이나 융자(담보대출)받을 수 잇나요?현ㅈㅐ 시댁에서 살고 있습니다5개월된 아이가 잇고 뱃속에 임신중인데저도 모르게 스트레스가 쌓여자꾸 출혈이 있네요.. 한시라도 빨리나가고 싶은데 방법있을까요?

   ▶ 질문
500으로 신혼부부전세대출이나LH전세대출이나 융자(담보대출)받을 수 잇나요?현ㅈㅐ 시댁에서 살고 있습니다5개월된 아이가 잇고 뱃속에 임신중인데저도 모르게 스트레스가 쌓여자꾸 출혈이 있네요.. 한시라도 빨리나가고 싶은데 방법있을까요?

   ▶ 응답
<응답1> 전세 담보 상담할수 있는 대출 상품은 있습니다.​정식등록된 등록허가업체로 여러업체 조건먼저 확인하시고 질문자님께 유리한 곳으로 선정하여​진행하시면 승인률이 높으시니 등록허가업체로 조건 검토해보세요​안전한 업체에서 상담이 중요하시오니 참고하시고, ​사전에 등록허가업체 여부는 필히 체크하시면 좋습니다.

<응답2> 전세 담보 알아보실수 있는 대출 상품은 있습니다.​대출을 진행하는데있어서 주의하셔야될점은 선입금및 카드 통장 ​수수료요구하는곳등은 사기니 주의하시고 신중하게 대출을​알아보실것을 권장해드립니다.​안전하게 허가업체에서 조건과 한도를 검토 하시고 결정하시는게 중요합니다.

------------------------------------------------------------------------------------